In [1]:
import matplotlib.pyplot as plt

In [2]:
import findspark
import timeit
findspark.init("/u/cs451/packages/spark")
from pyspark.sql import SparkSession
import random
spark = SparkSession.builder.appName("Project").master("local[2]").config('spark.ui.port', random.randrange(4000,5000)).getOrCreate()
from pyspark.sql import SQLContext
sc = spark.sparkContext
sqlcontext = SQLContext(sc)

In [3]:
import pandas as pd
from pyspark.sql.types import IntegerType, FloatType
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [4]:
def performPreProcessing(df):  
    #casting year, month and day to integer values
    df = df.withColumn("year", df["year"].cast(IntegerType()))
    df = df.withColumn("mo", df["mo"].cast(IntegerType()))
    df = df.withColumn("da", df["da"].cast(IntegerType()))
    df = df.withColumn("temp", df["temp"].cast(FloatType()))
    df = df.withColumn("slp", df["slp"].cast(FloatType()))
    df = df.withColumn("stp", df["stp"].cast(FloatType()))
    df = df.withColumn("wdsp", df["wdsp"].cast(FloatType()))
    df = df.withColumn("mxpsd", df["mxpsd"].cast(FloatType()))
    df = df.withColumn("max", df["max"].cast(FloatType()))
    df = df.withColumn("min", df["min"].cast(FloatType()))
    return df

In [5]:
def load_datasets_and_set_views():
    global stations, gsod_data_2009, gsod_data_2010, gsod_data_2011, gsod_data_2012, gsod_data_2013, gsod_data_2014, gsod_data_2015, gsod_data_2016, gsod_data_2017, gsod_data_2018, gsod_data_2019

    gsod_data_2009 = spark.read.format("csv").option("header", "true").load("/tmp/Project/gsod2009.txt").cache()
#     gsod_data_2009=performPreProcessing(gsod_data_2009)
    gsod_data_2009.createOrReplaceTempView("gsod_data_2009")

    gsod_data_2010 = spark.read.format("csv").option("header", "true").load("/tmp/Project/gsod2010.txt").cache()
#     gsod_data_2010 = performPreProcessing(gsod_data_2010)
    gsod_data_2010.createOrReplaceTempView("gsod_data_2010")

    gsod_data_2011 = spark.read.format("csv").option("header", "true").load("/tmp/Project/gsod2011.txt").cache()
#     gsod_data_2011 =performPreProcessing(gsod_data_2011)
    gsod_data_2011.createOrReplaceTempView("gsod_data_2011")

    gsod_data_2012 = spark.read.format("csv").option("header", "true").load("/tmp/Project/gsod2012.txt").cache()
#     gsod_data_2012 =performPreProcessing(gsod_data_2012)
    gsod_data_2012.createOrReplaceTempView("gsod_data_2012")

    gsod_data_2013 = spark.read.format("csv").option("header", "true").load("/tmp/Project/gsod2013.txt").cache()
#     gsod_data_2013 =performPreProcessing(gsod_data_2013)
    gsod_data_2013.createOrReplaceTempView("gsod_data_2013")

    gsod_data_2014 = spark.read.format("csv").option("header", "true").load("/tmp/Project/gsod2014.txt").cache()
#     gsod_data_2014 =performPreProcessing(gsod_data_2014)
    gsod_data_2014.createOrReplaceTempView("gsod_data_2014")

    gsod_data_2015 = spark.read.format("csv").option("header", "true").load("/tmp/Project/gsod2015.txt").cache()
#     gsod_data_2015 =performPreProcessing(gsod_data_2015)
    gsod_data_2015.createOrReplaceTempView("gsod_data_2015")

    gsod_data_2016 = spark.read.format("csv").option("header", "true").load("/tmp/Project/gsod2016.txt").cache()
#     gsod_data_2016 =performPreProcessing(gsod_data_2016)
    gsod_data_2016.createOrReplaceTempView("gsod_data_2016")

    gsod_data_2017 = spark.read.format("csv").option("header", "true").load("/tmp/Project/gsod2017.txt").cache()
#     gsod_data_2017 =performPreProcessing(gsod_data_2017)
    gsod_data_2017.createOrReplaceTempView("gsod_data_2017")

    gsod_data_2018 = spark.read.format("csv").option("header", "true").load("/tmp/Project/gsod2018.txt").cache()
#     gsod_data_2018 =performPreProcessing(gsod_data_2018)
    gsod_data_2018.createOrReplaceTempView("gsod_data_2018")

    gsod_data_2019 = spark.read.format("csv").option("header", "true").load("/tmp/Project/gsod2019.txt").cache()
#     gsod_data_2019 =performPreProcessing(gsod_data_2019)
    gsod_data_2019.createOrReplaceTempView("gsod_data_2019")

    stations = spark.read.format("csv").option("header", "true").load("/tmp/Project/stations.txt").cache()
    stations.createOrReplaceTempView("stations")

    print("done")

In [6]:
load_datasets_and_set_views()

done


In [ ]:
rmse_lr_train=[]
rmse_glr_train=[]
rmse_lr_test=[]
rmse_glr_test=[]
for month in ["01","02","03","04","05","06","07","08","09","10","11","12"]:

    print(month,".................\n")
    # month="01"
    country="US"

    query_2009=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2009 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2010=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2010 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2011=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2011 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2012=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2012 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2013=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2013 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2014=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2014 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2015=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2015 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2016=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2016 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2017=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2017 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2018=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2018 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2019=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2019 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))



    ######################################################

    query_2009 =performPreProcessing(query_2009)
    query_2010 =performPreProcessing(query_2010)
    query_2011 =performPreProcessing(query_2011)
    query_2012 =performPreProcessing(query_2012)
    query_2013 =performPreProcessing(query_2013)
    query_2014 =performPreProcessing(query_2014)
    query_2015 =performPreProcessing(query_2015)
    query_2016 =performPreProcessing(query_2016)
    query_2017 =performPreProcessing(query_2017)
    query_2018 =performPreProcessing(query_2018)


    

    ### Table Union

    df_all=query_2017.union(query_2018)\
#                     .union(query_2011)\
#                     .union(query_2012)\
#                     .union(query_2013)\
#                     .union(query_2014)\
#                     .union(query_2015)\
#                     .union(query_2016)\
#                     .union(query_2017)\
#                     .union(query_2018)\
#                     .union(query_2019)

    
    ##Monthly Average/Max/Min Temperature
    df_min=df_all.groupBy('stn').agg({'temp':'min'})
    df_max=df_all.groupBy('stn').agg({'temp':'max'})
    df_avg=df_all.groupBy('stn').agg({'temp':'avg'})
    # df_all.show(5)

    ### Feature Selection for Regression
    

    from pyspark.sql.functions import col
    feat_cols_reg=[ 'year', 'mo', 'da', 'slp', 'stp', 'wdsp','max','min']
    # dataset=sqlcontext.sql("SELECT g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2018 g")
    dataset_reg=df_all

    dataset_reg=dataset_reg.select(*(col(c).cast("double").alias(c) for c in dataset_reg.columns))


    vec_assembler_reg=VectorAssembler(inputCols= feat_cols_reg,outputCol='features')
    final_data_reg=vec_assembler_reg.transform(dataset_reg)
    final_data_reg=final_data_reg.select(['features','temp'])
    training_data_reg,test_data_reg=final_data_reg.randomSplit([0.75,0.25],seed=100)


    ### Feature Selection for Regression

    from pyspark.ml.regression import LinearRegression
    from pyspark.ml.evaluation import RegressionEvaluator


    start = timeit.timeit()

    lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8,labelCol='temp')

    # Fit the model
    model = lr.fit(training_data_reg)

    # Make predictions.
    predictions = model.transform(test_data_reg)
    pred_train = model.transform(training_data_reg)
    end = timeit.timeit()
#     print("Execution time Linear Regression 2009-10",end - start)
    # Select example rows to display.
#     predictions.select("prediction", "temp", "features").show(5)

    # Select (prediction, true label) and compute test error
    evaluator = RegressionEvaluator(
        labelCol="temp", predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(predictions)
    rmse_lr_test.append(rmse)
    print("Test RMSE for {}-Linear Regression".format(month), rmse)



    ### Generalized Linear Regression

    from pyspark.ml.regression import GeneralizedLinearRegression
    from pyspark.ml.evaluation import RegressionEvaluator

    # Load training data
    # dataset = spark.read.format("libsvm")\
    #     .load("data/mllib/sample_linear_regression_data.txt")

    start= timeit.timeit()
    glr = GeneralizedLinearRegression(labelCol='temp',family="gaussian", link="identity", maxIter=10, regParam=0.3)

    # Fit the model
    model = glr.fit(training_data_reg)

    # Make predictions.
    predictions = model.transform(test_data_reg)
    pred_train= model.transform(training_data_reg)
    end=timeit.timeit()

#     print("Execution time Generalized Linear Regression 2019-10",end - start)

    # Select example rows to display.
#     predictions.select("prediction", "temp", "features").show(5)

    # Select (prediction, true label) and compute test error
    evaluator = RegressionEvaluator(
        labelCol="temp", predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(predictions)
    rmse_glr_test.append(rmse)
    print("Test RMSE for {}-GLR".format(month), rmse,"\n\n")

print("Test rmse month wise Linear Regression",rmse_lr_test)
print("Test rmse month wise GLR",rmse_glr_test)


In [7]:
rmse_lr_train=[]
rmse_glr_train=[]
rmse_lr_test=[]
rmse_glr_test=[]
for month in ["01","02","03","04","05","06","07","08","09","10","11","12"]:

    print(month,".................\n")
    # month="01"
    country="US"

    query_2009=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2009 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2010=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2010 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2011=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2011 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2012=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2012 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2013=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2013 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2014=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2014 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2015=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2015 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2016=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2016 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2017=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2017 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2018=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2018 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

    query_2019=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2019 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))



    ######################################################

    query_2009 =performPreProcessing(query_2009)
    query_2010 =performPreProcessing(query_2010)
    query_2011 =performPreProcessing(query_2011)
    query_2012 =performPreProcessing(query_2012)
    query_2013 =performPreProcessing(query_2013)
    query_2014 =performPreProcessing(query_2014)
    query_2015 =performPreProcessing(query_2015)
    query_2016 =performPreProcessing(query_2016)
    query_2017 =performPreProcessing(query_2017)
    query_2018 =performPreProcessing(query_2018)


    

    ### Table Union

    df_all=query_2009.union(query_2010)\
                    .union(query_2011)\
                    .union(query_2012)\
                    .union(query_2013)\
                    .union(query_2014)\
                    .union(query_2015)\
                    .union(query_2016)\
                    .union(query_2017)\
                    .union(query_2018)\
                    .union(query_2019)

    
    ##Monthly Average/Max/Min Temperature
    df_min=df_all.groupBy('stn').agg({'temp':'min'})
    df_max=df_all.groupBy('stn').agg({'temp':'max'})
    df_avg=df_all.groupBy('stn').agg({'temp':'avg'})
    # df_all.show(5)

    ### Feature Selection for Regression
    

    from pyspark.sql.functions import col
    feat_cols_reg=[ 'year', 'mo', 'da', 'slp', 'stp', 'wdsp']
    # dataset=sqlcontext.sql("SELECT g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2018 g")
    dataset_reg=df_all

    dataset_reg=dataset_reg.select(*(col(c).cast("double").alias(c) for c in dataset_reg.columns))


    vec_assembler_reg=VectorAssembler(inputCols= feat_cols_reg,outputCol='features')
    final_data_reg=vec_assembler_reg.transform(dataset_reg)
    final_data_reg=final_data_reg.select(['features','temp'])
    training_data_reg,test_data_reg=final_data_reg.randomSplit([0.75,0.25],seed=100)


    ### Feature Selection for Regression

    from pyspark.ml.regression import LinearRegression
    from pyspark.ml.evaluation import RegressionEvaluator


    start = timeit.timeit()

    lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8,labelCol='temp')

    # Fit the model
    model = lr.fit(training_data_reg)

    # Make predictions.
    predictions = model.transform(test_data_reg)
    pred_train = model.transform(training_data_reg)
    end = timeit.timeit()
#     print("Execution time Linear Regression 2009-10",end - start)
    # Select example rows to display.
#     predictions.select("prediction", "temp", "features").show(5)

    # Select (prediction, true label) and compute test error
    evaluator = RegressionEvaluator(
        labelCol="temp", predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(predictions)
    rmse_lr_test.append(rmse)
    print("Test RMSE for {}-Linear Regression".format(month), rmse)



    ### Generalized Linear Regression

    from pyspark.ml.regression import GeneralizedLinearRegression
    from pyspark.ml.evaluation import RegressionEvaluator

    # Load training data
    # dataset = spark.read.format("libsvm")\
    #     .load("data/mllib/sample_linear_regression_data.txt")

    start= timeit.timeit()
    glr = GeneralizedLinearRegression(labelCol='temp',family="gaussian", link="identity", maxIter=10, regParam=0.3)

    # Fit the model
    model = glr.fit(training_data_reg)

    # Make predictions.
    predictions = model.transform(test_data_reg)
    pred_train= model.transform(training_data_reg)
    end=timeit.timeit()

#     print("Execution time Generalized Linear Regression 2019-10",end - start)

    # Select example rows to display.
#     predictions.select("prediction", "temp", "features").show(5)

    # Select (prediction, true label) and compute test error
    evaluator = RegressionEvaluator(
        labelCol="temp", predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(predictions)
    rmse_glr_test.append(rmse)
    print("Test RMSE for {}-GLR".format(month), rmse,"\n\n")

print("Test rmse month wise Linear Regression",rmse_lr_test)
print("Test rmse month wise GLR",rmse_glr_test)


('01', '.................\n')
('Test RMSE for 01-Linear Regression', 19.82529838241916)
('Test RMSE for 01-GLR', 19.811872517036257, '\n\n')
('02', '.................\n')
('Test RMSE for 02-Linear Regression', 19.32978177367141)
('Test RMSE for 02-GLR', 19.31680842430419, '\n\n')
('03', '.................\n')
('Test RMSE for 03-Linear Regression', 19.972827452569224)
('Test RMSE for 03-GLR', 19.958497124987378, '\n\n')
('04', '.................\n')
('Test RMSE for 04-Linear Regression', 15.653844602306176)
('Test RMSE for 04-GLR', 15.646385229670612, '\n\n')
('05', '.................\n')
('Test RMSE for 05-Linear Regression', 12.655812612895145)
('Test RMSE for 05-GLR', 12.63234714619344, '\n\n')
('06', '.................\n')
('Test RMSE for 06-Linear Regression', 11.3433329694773)
('Test RMSE for 06-GLR', 11.31837189717347, '\n\n')
('07', '.................\n')
('Test RMSE for 07-Linear Regression', 10.467897821983168)
('Test RMSE for 07-GLR', 10.442470699551729, '\n\n')
('08', '.....

In [7]:
rmse_lr=[]
rmse_glr=[]
month="01"
country="US"

In [8]:
query_2009=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2009 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

query_2010=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2010 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

query_2011=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2011 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

query_2012=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2012 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

query_2013=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2013 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

query_2014=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2014 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

query_2015=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2015 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

query_2016=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2016 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

query_2017=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2017 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

query_2018=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2018 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

query_2019=spark.sql("SELECT s.country, g.stn, g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2019 g, stations s where s.usaf=g.stn AND g.mo={0} AND s.country='{1}' AND g.slp!=9999.9 AND g.stp!=9999.9 AND g.wdsp!=999.9 AND g.mxpsd!=999.9 AND g.max!=9999.9 AND g.min!=9999.9".format(month,country))

In [9]:
query_2009 =performPreProcessing(query_2009)
query_2010 =performPreProcessing(query_2010)
query_2011 =performPreProcessing(query_2011)
query_2012 =performPreProcessing(query_2012)
query_2013 =performPreProcessing(query_2013)
query_2014 =performPreProcessing(query_2014)
query_2015 =performPreProcessing(query_2015)
query_2016 =performPreProcessing(query_2016)
query_2017 =performPreProcessing(query_2017)
query_2018 =performPreProcessing(query_2018)

In [12]:
df_all=query_2017.union(query_2018)\
#                 .union(query_2011)\
#                 .union(query_2012)\
#                 .union(query_2013)\
#                 .union(query_2014)\
#                 .union(query_2015)\
#                 .union(query_2016)\
#                 .union(query_2017)\
#                 .union(query_2018)\
#                 .union(query_2019)

In [13]:
##Monthly Average Temperature
df_min=df_all.groupBy('stn').agg({'temp':'min'})
df_max=df_all.groupBy('stn').agg({'temp':'max'})
df_avg=df_all.groupBy('stn').agg({'temp':'avg'})
# df_all.show(5)

### Feature Selection for Regression

In [14]:
from pyspark.sql.functions import col
feat_cols_reg=[ 'year', 'mo', 'da', 'slp', 'stp', 'wdsp', 'mxpsd', 'max', 'min']
# dataset=sqlcontext.sql("SELECT g.year, g.mo, g.da, g.temp, g.slp, g.stp, g.wdsp, g.mxpsd, g.max, g.min, g.rain_drizzle FROM gsod_data_2018 g")
dataset_reg=df_all

dataset_reg=dataset_reg.select(*(col(c).cast("double").alias(c) for c in dataset_reg.columns))

In [15]:
vec_assembler_reg=VectorAssembler(inputCols= feat_cols_reg,outputCol='features')
final_data_reg=vec_assembler_reg.transform(dataset_reg)
final_data_reg=final_data_reg.select(['features','temp'])
training_data_reg,test_data_reg=final_data_reg.randomSplit([0.75,0.25],seed=100)

### Linear Regression

In [18]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator


start = timeit.timeit()

lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8,labelCol='temp')

# Fit the model
model = lr.fit(training_data_reg)

# Make predictions.
predictions = model.transform(test_data_reg)
pred_train = model.transform(training_data_reg)
end = timeit.timeit()
print("Execution time Linear Regression 2017-18",end - start)
# Select example rows to display.
predictions.select("prediction", "temp", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="temp", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Test RMSE for month-Linear Regression".format(month), rmse)

('Execution time Linear Regression 2017-18', 0.010365962982177734)
+------------------+------------------+--------------------+
|        prediction|              temp|            features|
+------------------+------------------+--------------------+
|12.145587618247177|13.300000190734863|[2017.0,1.0,1.0,1...|
|31.999954649711942|31.700000762939453|[2017.0,1.0,1.0,1...|
| 36.67853119977634|              36.0|[2017.0,1.0,1.0,1...|
| 36.67853119977634|              36.0|[2017.0,1.0,1.0,1...|
| 33.61866391700639|33.900001525878906|[2017.0,1.0,1.0,1...|
+------------------+------------------+--------------------+
only showing top 5 rows

('Test RMSE for month-Linear Regression', 1.5868863764744965)


### Generalized Linear Regression

In [19]:
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Load training data
# dataset = spark.read.format("libsvm")\
#     .load("data/mllib/sample_linear_regression_data.txt")

start= timeit.timeit()
glr = GeneralizedLinearRegression(labelCol='temp',family="gaussian", link="identity", maxIter=10, regParam=0.3)

# Fit the model
model = glr.fit(training_data_reg)

# Make predictions.
predictions = model.transform(test_data_reg)
pred_train= model.transform(training_data_reg)
end=timeit.timeit()

print("Execution time Generalized Linear Regression 2017-18",end - start)

# Select example rows to display.
predictions.select("prediction", "temp", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="temp", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Test RMSE for month-Linear Regression".format(month), rmse)



('Execution time Generalized Linear Regression 2017-18', 0.0014338493347167969)
+------------------+------------------+--------------------+
|        prediction|              temp|            features|
+------------------+------------------+--------------------+
|11.838353256453757|13.300000190734863|[2017.0,1.0,1.0,1...|
| 32.43066880285326|31.700000762939453|[2017.0,1.0,1.0,1...|
| 36.71637400035709|              36.0|[2017.0,1.0,1.0,1...|
| 36.71637400035709|              36.0|[2017.0,1.0,1.0,1...|
|33.653294848642474|33.900001525878906|[2017.0,1.0,1.0,1...|
+------------------+------------------+--------------------+
only showing top 5 rows

('Test RMSE for month-Linear Regression', 1.5560629094967058)
